In [7]:
import pandas
 
positive_pips = 30
negative_pips = 10
point = 0.0001
 
# ファイルはここからダウンロードしている
# http://www.histdata.com/download-free-forex-historical-data/?/metatrader/1-minute-bar-quotes/EURUSD
 
df = pandas.read_csv('./dataset/DAT_MT_EURUSD_M1_201807.csv',
                     names=['date', 'time', 'o', 'h', 'l', 'c', 'v'],
                     parse_dates={'datetime': ['date', 'time']},
                     )  # type DataFrame
df.index = df['datetime']
# print(df[1707:1950])
df['signal'] = 0
print(len(df))
 
for i in range(len(df) - 241):
    base = df.iloc[i]
    target = df[i:i + 240]
    upper_negative = target.loc[target['h'] > df['h'][i] + negative_pips * point]
    upper_positive = target.loc[target['h'] > df['h'][i] + positive_pips * point]
    lower_negative = target.loc[target['l'] < df['l'][i] - negative_pips * point]
    lower_positive = target.loc[target['l'] < df['l'][i] - positive_pips * point]
 
    if len(upper_positive) > 0:
        if len(lower_negative) == 0:
            df['signal'][i] = 1
        elif upper_positive.iloc[0]['datetime'] < lower_negative.iloc[0]['datetime']:
            df['signal'][i] = 1
    if len(lower_positive) > 0:
        if len(upper_negative) == 0:
            df['signal'][i] = -1
        elif lower_positive.iloc[0]['datetime'] < upper_negative.iloc[0]['datetime']:
            df['signal'][i] = -1
 
df.dropna()
df.to_csv('EURUSD_M1_201807_with_signal.csv')

14366


/Users/noriakioshita/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/noriakioshita/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/noriakioshita/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/noriakioshita/.pyenv/versions/

In [8]:
import numpy
import pandas
 
df = pandas.read_csv('EURUSD_M1_201807_with_signal.csv')
df.drop('datetime', axis=1)
maxlen = 1440  # ひとつの時系列データの長さ(24時間)
 
x = numpy.empty((0, 1440), float)
y = numpy.empty((0, 1), int)
df.drop('v', axis=1)
signal = df['signal']
df.drop('signal', axis=1)
 
# 1時間ごとにトレーニングデータを作る
for i in range(1440, len(df), 60):
    # とりあえず終値だけにしてみる
    append_x = numpy.array([df['c'].loc[i - maxlen: i - 1]])
    append_y = numpy.array([0, 0, 0])
 
    # 1-of-K 表現に変換
    if signal.loc[i] < 0:
        append_y[0] = 1
    elif signal.loc[i] > 0:
        append_y[2] = 1
    else:
        append_y[1] = 1
 
    x = numpy.append(x, append_x)
    y = numpy.append(y, append_y)
 
numpy.save('EURUSD_M1_201807_x.npy', x)
numpy.save('EURUSD_M1_201807_y.npy', y)

In [9]:
import numpy
from keras.callbacks import EarlyStopping, TensorBoard
from keras.layers import GRU, Dense, Activation
from keras.models import Sequential
from keras.optimizers import Adam
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
 
 
def weight_variable(shape, name=None):
    return numpy.random.normal(scale=.01, size=shape)
 
 
maxlen = 1440  # ひとつの時系列データの長さ(24時間)
x = numpy.load('EURUSD_M1_201807_x.npy')
y = numpy.load('EURUSD_M1_201807_y.npy')
 
# 正規化
x = x / numpy.linalg.norm(x)
 
x = x.reshape((int(len(x) / maxlen), maxlen, 1))
y = y.reshape((len(x), 3))
 
X_train, X_test, Y_train, Y_test = train_test_split(x, y, train_size=0.8)
X_train, X_validation, Y_train, Y_validation = \
    train_test_split(X_train, Y_train, train_size=0.8)
 
n_in = len(x[0][0])  # 1
n_hidden = 16
n_out = len(y[0])  # 3
 
epochs = 10
batch_size = 10
 
early_stopping = EarlyStopping(monitor='loss', patience=100, verbose=1)
 
model = Sequential()
model.add(GRU(n_hidden,
              kernel_initializer=weight_variable,
              input_shape=(maxlen, n_in)))
model.add(Dense(n_out))
model.add(Activation('softmax'))
 
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)
 
hist = model.fit(X_train, Y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 validation_data=(X_validation, Y_validation),
                 callbacks=[early_stopping, TensorBoard(log_dir="log", histogram_freq=1)])
 
'''
予測精度の評価
'''
loss_and_metrics = model.evaluate(X_test, Y_test)
print(loss_and_metrics)

/Users/noriakioshita/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/noriakioshita/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Train on 137 samples, validate on 35 samples
Epoch 1/10
137/137 [==============================] - 12s 84ms/step - loss: 1.0760 - val_loss: 1.0364
Epoch 2/10
137/137 [==============================] - 11s 82ms/step - loss: 1.0149 - val_loss: 0.9553
Epoch 3/10
137/137 [==============================] - 12s 84ms/step - loss: 0.9357 - val_loss: 0.8396
Epoch 4/10
137/137 [==============================] - 11s 83ms/step - loss: 0.8287 - val_loss: 0.6634
Epoch 5/10
137/137 [==============================] - 11s 81ms/step - loss: 0.7122 - val_loss: 0.4846
Epoch 6/10
137/137 [==============================] - 11s 83ms/step - loss: 0.6432 - val_loss: 0.4780
Epoch 7/10
137/137 [==============================] - 11s 82ms/step - loss: 0.6442 - val_loss: 0.4823
Epoch 8/10
137/137 [==============================] - 11s 81ms/step - loss: 0.6453 - val_loss: 0.4726
Epoch 9/10
137/137 [==============================] - 11s 80ms/step - loss: 0.6461 - val_loss: 0.4925
Epoch 10/10
44/44 [==================